# Basalt synthetic

An example of creating a simple land elastic synthetic. First a function for plotting.

In [ ]:
import numpy as np
import holoviews as hv
def createPlot(data,p1,p2,p3):
    if isinstance(data,np.ndarray):
        fld=data
        hyper=syntheticModel.hypercube(ns=[data.shape[2],data.shape[1],data.shape[0]])
    else:
        fld=data.getNdArray()
        hyper=data.getHyper()
    ax1=hyper.axes[0]
    ax2=hyper.axes[1]
    ax3=hyper.axes[2]
    mn=fld.min()
    mx=fld.max()
    front=fld[p3,:,::-1].T
    top=fld[::-1,:,p1]
    side=fld[:,p2,::-1].T
    b=(ax1.o,ax1.o+ax1.d*(ax1.n-1),ax2.o,ax2.o+ax2.d*(ax2.n-1))
    frontF=hv.Image(front,kdims=["X","Z"],bounds=(ax2.o,ax1.o,ax2.o+ax2.d*(ax2.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,invert_yaxis=True,aspect=1,axiswise=True)
    sideF=hv.Image(side,kdims=["Y","Z"],bounds=(ax3.o,ax1.o,ax3.o+ax3.d*(ax3.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,yaxis=None,invert_yaxis=True,aspect=1.,axiswise=True)
    topF=hv.Image(top,kdims=["X","Y"],bounds=(ax2.o,ax2.o,ax2.o+ax2.d*(ax2.n-1),ax3.o+ax3.d*(ax3.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,xaxis=None,aspect=1,axiswise=True)
    botF=hv.Layout(frontF+sideF).opts(toolbar=None)
    tot=hv.Layout(topF+hv.Empty()+frontF+sideF).cols(2).opts(toolbar=None)
    return tot
    
    

We will begin by installing a basement at 5500 m/s.

In [ ]:
import syntheticModel
import holoviews as hv
hv.extension("bokeh","matplotlib",logo=False)

hyper=syntheticModel.hypercube(ns=[20,800,800],os=[0,0,0],ds=[5,5,5])
basement=syntheticModel.geoModel(hyper,{"vp":3000.},"vp")

Lets define several different types of deposity functions.

In [ ]:
vsPars={}
rhoPars={}
depositC=syntheticModel.deposit(**vsPars,**rhoPars)  #Constant velocity
depositI=syntheticModel.deposit(interbedThick=20.,interbedThickDev=13.,interbedPropDev=100.,**vsPars,**rhoPars) #Interbeded layer
depositN=syntheticModel.deposit(noiseDev=300.,**vsPars,**rhoPars) #Add some noise to constant velocity\
depositIC=syntheticModel.deposit(interbedThick=15.,interbedThickDev=13.,interbedPropDev=100.,interbedDev=4.,**vsPars,**rhoPars) #Interbed with noise
depositICN=syntheticModel.deposit(noiseDev=120,interbedThick=15.,\
                                  interbedThickDev=13.,noiseFrequency=3,noiseZStretch=10,interbedPropDev=100.,
                                  interbedDev=4.,**vsPars,**rhoPars)

In [ ]:
from operator import iadd
import opensimplex
import datetime
import numba
import numpy as np

@numba.njit(parallel=True)
def scaleVec(x,sc):
    xuse=x.copy()
    for i in numba.prange(x.shape[0]):
        xuse[i]=x[i]*sc
    return xuse

@numba.njit(parallel=True)
def addMat3(outA,inA,sc):
    for i3 in numba.prange(inA.shape[0]):
        for i2 in range(inA.shape[1]):
            for i1 in range(inA.shape[2]):
                outA[i3,i2,i1]+=inA[i3,i2,i1]*sc
    


class simplexNoise:
    """Interface for opensimplex"""
    
    def __init__(self,seed:int=None):
        if seed is None:
            seed=int(datetime.datetime.utcnow().timestamp())
        self.seed=seed
    
    def noise2D(self,x:np.ndarray,y:np.ndarray,n_octaves:int=3,frequency:float=1.0, \
      amplitude:float=1.0,lucanarity:float=2.,persistence:float=0.5)->np.ndarray:
    
        amps=[amplitude]
        freqs=[frequency]
        for i in range(n_octaves-1):
            amps.append(amps[i]*persistence)
            freqs.append(freqs[i]*lucanarity)
        
        return self.noise2DF(x,y,freqs,amps)
        
    def noise2DF(self,x:np.ndarray,y:np.ndarray,freqs:list,amps:list)->np.ndarray:
        
        ar=np.zeros((y.shape[0],x.shape[0]))
        for i in range(len(freqs)):
            opensimplex.seed(self.seed)
            self.seed+=1
            ar+=opensimplex.noise2array(x*4*freqs[i],y*4*freqs[i])*amps[i]
        return ar
        
    def noise3D(self,x:np.ndarray,y:np.ndarray,z:np.ndarray,frequency:float=1.0, \
      amplitude:float=1.0,n_octaves:int=3,persistence:float=0.5,lucanarity:float=2.)->np.ndarray:
    
        amps=[amplitude]
        freqs=[frequency]
        for i in range(n_octaves-1):
            amps.append(amps[i]*persistence)
            freqs.append(freqs[i]*lucanarity)
        
        return self.noise3DF(x,y,z,freqs,amps)

    def noise3DF(self,x:np.ndarray,y:np.ndarray,z:np.ndarray,freqs:list,amps:list)->np.ndarray:
        
        ar=np.zeros((z.shape[0],y.shape[0],x.shape[0]))
        for i in range(len(freqs)):
            opensimplex.seed(self.seed)
            self.seed+=1
            xu=scaleVec(x,4*freqs[i])
            yu=scaleVec(y,4*freqs[i])
            zu=scaleVec(z,4*freqs[i])
            xt=opensimplex.noise3array(xu,yu,zu)
            addMat3(ar,xt,amps[i])
        return ar   

In [ ]:
from numpy.core.numeric import _isclose_dispatcher
import syntheticModel.syntheticModel as syntheticModel
import syntheticModel.simplexNoise as simplexNoise
import numpy as np
import numba
import random
import math
import syntheticModel.event as event
import datetime
import syntheticModel.sincTable as sincTable
debugBounds=False
@numba.njit()
def calc1DDeposition(n1:int,thickness):
  """
  Calcualte a 1-D property function
  
  n1 - Number of samples
  thickness - 1-D array of thicknesses

  Returns:

  layers - 1-D Array layer number
  outVals   - 1-D Array with values
  
  """
  layers=np.zeros((n1,),dtype=np.float32)
  curThick=0
  ilayer=0
  for i in range(n1):
    curThick+=1
    if thickness[ilayer] < curThick:
      layers[ilayer]=i
      ilayer+=1
      curThick=0

  ilayer+=1
  return layers,ilayer



@numba.njit(parallel=True,boundscheck=debugBounds)
def getHeight(inA):
  """
  Find the height of the bottom of the undefined top layer
  
  inA - Input array
  
  """
  height=np.zeros((inA.shape[0],inA.shape[1]),dtype=np.int32)
  for i3 in numba.prange(inA.shape[0]):
    for i2 in range(inA.shape[1]):
      height[i3,i2]=0
      while inA[i3,i2,height[i3,i2]]==-1 and height[i3,i2] <inA.shape[2]-1:
        height[i3,i2]+=1

  return height



@numba.njit(parallel=True)
def applyNoise(in3D,noise,dev):
  """
  Apply noise to an array

  in3D - Input Array
  noise - Noise array
  dev - StdDev for noise
  
  
  """
  out=noise.copy()
  for i3 in numba.prange(noise.shape[0]):
    for i2 in range(noise.shape[1]):
      for i1 in range(noise.shape[2]):
        out[i3,i2,i1]=in3D[i3,i2,i1]+noise[i3,i2,i1]*dev

  return out

@numba.njit(parallel=True,boundscheck=debugBounds)
def modifyRGT(rgt,height,ievent):
  """
  Update RGT field

  rgt - RGT field
  height - Height at various locations
  ievent - Event 
 
  
  """
  for i3 in numba.prange(rgt.shape[0]):
    for i2 in range(rgt.shape[1]):
      b0=ievent
      d=1./float(height[i3,i2])
      for i1 in range(rgt.shape[2]):
          rgt[i3,i2,i1]=b0+d*i1



def getParams(param:str, kw:dict)->(int,dict):
  """"
  Search parameters for all correpsonding to specified parameter
  
   param- Parameter to look for
   kw - Input dictionary
  """
      

  pars={ "dev":0.,"noiseFreq":1.,"noiseLacunarity":2.,"noisePersistence":.5,"noiseOctaves":3,\
       "noiseAbs":False,"ratio":1.,"depthGrad":0.,"noiseZStretch":1.,"noiseDev":0.,\
         "interbedPropDev":0.,"prop":2500,"layer":None}
  kout={}
  for k,v in pars.items():
    if "%s_%s"%(param,k) in kw:
      kout[k]=kw["%s_%s"%(param,k)]
    elif k in kw:
      kout[k]=kw[k]
    else:
      kout[k]=pars[k]
  if f"{param}_prop" in kw:
    return 2,kout
  elif f"{param}_ratio" in kw:
    kout["ratio"]=kw[f"{param}_ratio"]
    return 1, kout
  else:
    return 0, kout
   

@numba.njit(parallel=True,boundscheck=debugBounds)
def createPropArray(nlayer,grad,noise3D,layer1D,val1D,noiseV,izOff):
    """"
    Create the property array field

    nlayer - Number of layers
    grad.   - gradient
    noise3D - Layer boundary noise
    layer1D - 1-D layer boundaries
    val1D   - Property values
    noiseV  - SPatial noise
    izOff   - Random array of locations to begin grabing from noiseV


    """
    print("In create prop")
    outA=noise3D.copy()

    for i3 in numba.prange(outA.shape[0]):
        for i2 in range(outA.shape[1]):
            ilast=0
            for i1 in range(nlayer):
                inext=max(0,min(outA.shape[2],int(.5+noise3D[i3,i2,i1]+layer1D[i1])))
                for i in range(ilast,inext):
                    outA[i3,i2,i]=val1D[i1]+grad*i+noiseV[i3,i2,i+izOff[i1]]
                ilast=inext
            for i in range(ilast,noise3D.shape[2]):
                outA[i3,i2,i]=val1D[nlayer-1]+grad*i+noiseV[i3,i2,i+izOff[nlayer-1]]  

    return outA

@numba.njit(parallel=True,boundscheck=debugBounds)
def setRGT(nlayer,noise3D,layer1D,val1D,noiseV,rgt,icur):
  """"
  Set RGT 

  nlayer - Number of layers
  noise3D - Layer boundary noise
  layer1D - 1-D layer boundaries
  val1D   - Property values
  noiseV  - SPatial noise
  rgt     - Time
  icur    - Current event
  
  """
  f0=icur
  df=1./float(nlayer)
  outA=noise3D.copy()
  for i3 in numba.prange(outA.shape[0]):
    for i2 in range(outA.shape[1]):
      ilast=0
      for i1 in range(nlayer):
        inext=max(0,min(outA.shape[2],int(.5+noise3D[i3,i2,i1]+layer1D[i1])))

        m0=f0+df*i1
        dm=df/float(inext-ilast)
        for i in range(ilast,inext):
          rgt[i3,i2,i]=m0+dm*i
        ilast=inext
     
      m0=f0+df*(nlayer-1)
      dm=df/float(noise3D.shape[2]-ilast)
      for i in range(ilast,noise3D.shape[2]):
        rgt[i3,i2,i]=m0+dm*i
          
  return outA
@numba.njit(parallel=True,boundscheck=debugBounds)
def modifyFloatArray(array,height,dep,sinc,expand):
  """"
  Modify float array inserting new deposition

  array - Array to be modified
  height - Size of deposition as  function  x,y
  dep - What to put into the depsotion
  sinc - Sinc table
  expand - Whether or not to expand
  
  """
  if expand:
    mn=height.min()
  for i3 in numba.prange(array.shape[0]):
    for i2 in range(array.shape[1]):
      tmp=np.zeros((dep.shape[2]+8),dtype=np.float32)
      for i in range(dep.shape[2]):
        tmp[4+i]=dep[i3,i2,i]
      for i in range(4):
        tmp[i]=tmp[4]
        tmp[tmp.shape[0]-4+i]=tmp[tmp.shape[0]-5]

      if expand:
        d=mn/height[i3,i2]
        #d=1.-float(height[i3,i2])/float(dep.shape[2])
      else:
        d=1.
      for i1 in range(height[i3,i2]):
        v=d*i1
        i=int(v)
        isinc=min(sinc.shape[0]-1,int((v-i)*10000+.5))
        array[i3,i2,i1]=0

        for j in range(8):
          array[i3,i2,i1]+=tmp[i+j]*sinc[isinc,j]


@numba.njit(parallel=True,boundscheck=debugBounds)
def modifyLayer(array,height,ievent):
  """
  array - Layer to modify
  height - Height as a function of x and y
  ievent - Event"""
  for i3 in numba.prange(array.shape[0]):
    for i2 in range(array.shape[1]):
      for i1 in range(height[i3,i2]):
        array[i3,i2,i1]=ievent


class deposit(event.event):
  """Base class for depositing"""
  def __init__(self,**kw):
      """Initialize the base class for deposition"""
      super().__init__(**kw)

  def applyBase(self,inM:syntheticModel.geoModel,thick:int=30,depthGrad:float=30.,\
     interbedThickAvg:float=7.5,interbedThickDev:float=0.,interbedPropDev:float=0.,\
     prop:float=2500.,expand:float=True,noiseFreq:float=1, noiseLacunarity:float=2., \
       noisePersistence:float=.5,noiseAbs:bool=False,noiseZStretch=1.,interbedOctaves=3,\
      interbedDev=0.,interbedFreq=2.,interbedLacunarity=2.,interbedPersistence=.5,\
      interbedZStretch=1.,seed=None,\
      noiseOctaves:int=3,noiseDev:float=0.,layer=None,**kw)->syntheticModel.geoModel:
      """Deposit a layer
      Arguments:
      modIn - Input geomodel
      seed  - [None] - Random seed,set for reproducibility
      thick - [30] - int; Thickness of the layer in number of samples
      depthGrad - [0.] - float; Gradient of primary parameter as a function of depth
      interbedThickAvg [7.5] - int; mean thickness of interbed layers in sample
      interbedThickDev  [0.] - int; std deviation of interbed layers  (stdDev)
      interbedPropDev [0.] Variation in interbed property  (stdDev)
      prop - [2500] Basic property value
      expand - [false] Whether or not expand (pinchout) or deposit horizontally
      layer - [None] floatTensor1D containing values for deposit
      noiseFreq - [1.] Higher frequency means more faster change
      noiseLacunarity [2.] How multiple octaves increase in frequency
      noisePeristence [.5] How multiple octaves max value changes 
      noiseZStretch [1.] Whether to stretch or compress Z axis to change noise psttern
      noseOctaves   [3] Number of noise octaves
      noiseAbs  [False] Whether or not to take the absolution value of the noise function
      noiseDev  [0.] StdDev to noise field
      interbedDev [0.] StDev of interbed layer boundaries
      interbedFreq     [2.] Higher frequency means faster change
      interbedLacunarity [2.] How multiple octaves increase in frequency
      interbedPeristence [.5] How multiple octaves max value changes 
      interbedZStretch  [1.]  How much to stretch Z axis when calculating layer boundary noise
      interbedOctaves [3] Number of octaves 
      
      
      Arguments that can be specified independantly for each parameter. For example, if you have a parameter
        'p1' to specify the deviation you would use 'p1_dev'.



      _noiseFreq - [2.] Higher frequency means more faster change
      _noiseLacunarity [2.] How multiple octaves increase in frequency
      _noisePeristence [.5] How multiple octaves max value changes 
      _noseOctaves   [3] Number of noise octaves
      _noiseAbs  [False] Whether or not to take the absolution value of the noise function
      _noiseZStretch [1.] How much to stretch z axis to create asymetrical noise

      Method 1:
      _ratio - Specify A ratio between base propety and this property
      _dev   - Specify deviation from ratio      


      Method 2:
      _depthGrad - [0.] - float; Gradient of primary parameter as a function of depth
      _noiseDev -[0.]  - float; Spatial deviation for noise
      _interbedPropDev [0.] StdDev in interbed property 
      _prop - [2500] Basic property value
      _layer - [null] floatTensor1D containing values for deposit





      Returns
      output geomodel
      """
      if seed is None:
        seed =int(datetime.datetime.utcnow().timestamp())
      np.random.seed(seed)
      random.seed(seed)

      z=inM.getPrimaryProperty()
    
      axes=inM.getPrimaryProperty().getHyper().axes
      allArgs={**locals(),**kw}
      del allArgs["kw"]
      exts=[ax.d*ax.n for ax in axes]
      # cpp or python modes
      n1Use=inM.findMaxDeposit()+thick
      maxE=max(max(axes[0].d*n1Use,exts[1]),exts[2])
      if layer==None:
        layer1D,nlayer=calc1DDeposition(n1Use,np.random.normal(interbedThickAvg/axes[0].d,interbedThickDev/axes[0].d,\
            (n1Use,)))
        if abs(interbedPropDev)<0.0001:
            nlayer=1
        val1D=np.random.normal(prop,interbedPropDev,(n1Use,))
      else:
        if not isinstance(layer,np.ndarray):
            raise Exception("layer must be a floatTensor")
        val1D=layer
        layer1D=np.linspace(0,val1D.shape[0]-1,val1D.shape[0])
        nlayer=val1D.shape[0]
        if val1D.shape[0] >= n1Use:
          raise Exception("layer is not large enough must be at least %d",n1Use)
            
      outM=inM.expand(thick,0)
      height=getHeight(outM.getLayer().getNdArray())

      z=np.linspace(0,3.*n1Use*axes[0].d/maxE*noiseZStretch*4,4*n1Use,dtype=np.float32)
      y=np.linspace(0,3.*exts[1]/maxE,axes[1].n,dtype=np.float32)
      x=np.linspace(0.,3.*exts[2]/maxE,axes[2].n,dtype=np.float32)
      zL=np.linspace(0,3.*n1Use*axes[0].d/maxE*interbedZStretch,n1Use,dtype=np.float32)

      layerNG=simplexNoise.simplexNoise(random.randint(0,1000*1000*1000))
      noiseG=simplexNoise.simplexNoise(random.randint(0,1000*1000*1000)) 
      noiseV=noiseG.noise3D(z,y,x,frequency=noiseFreq,lucanarity=noiseLacunarity,\
        persistence=noisePersistence,n_octaves=noiseOctaves,amplitude=noiseDev)
      ievent=outM.getNewEvent()

      noise3D=layerNG.noise3D(zL,y,x,n_octaves=interbedOctaves,amplitude=interbedDev)
      tst=random.sample(range(n1Use,3*n1Use),nlayer)
      print(nlayer,n1Use)  
      sendList = numba.typed.List()
      [sendList.append(x) for x in tst]
      print(sendList,"gbefore ")
      baseValues=createPropArray(nlayer,depthGrad/axes[0].d,noise3D,layer1D,val1D,noiseV,\
        sendList)
      mainProp=inM.getPrimary()
      sinc=sincTable.table
      for fld in outM.getFloatFieldList():
          modify=False
          if fld==mainProp:
            modify=True
            noiseT=baseValues
          elif fld=="rgt": 
            rgt=outM.getFloatField("rgt")
            setRGT(nlayer,noise3D,layer1D,val1D,noiseV,rgt,icur)
          else:        
            method,kProp=getParams(fld,allArgs)
            noiseG=simplexNoise.simplexNoise(random.randint(0,1000*1000*1000))
            if method!=0:
              z=np.linspace(0,3.*n1Use*axes[0].d/maxE*kProp["noiseZStretch"],n1Use,dtype=np.float32)
              propV=noiseG.noise3D(z,y,x,n_octaves=kProp["noiseOctaves"],frequency=kProp["noiseFreq"],\
                lucanarity=kProp["noiseLacunarity"],persistence=kProp["noisePersistence"],\
                  amplitude=kProp["noiseDev"])
              if kProp["noiseAbs"]:
                propV=abs(propV)              
              modify=True
              if method==1:
                propV=noiseG.noise3D(z,y,x,n_octaves=kProp["noiseOctaves"],frequency=kProp["noiseFreq"],\
                lucanarity=kProp["noiseLacunarity"],persistence=kProp["noisePersistence"])          
                if kProp["noiseAbs"]:
                  propV=abs(propV)   
                baseS=baseValues*kProp["ratio"]
                noiseT=applyNoise(baseS,propV,kProp["dev"])
              elif method==2:
                propV=noiseG.noise3D(z,y,x,n_octaves=kProp["noiseOctaves"],frequency=kProp["noiseFreq"],\
                lucanarity=kProp["noiseLacunarity"],persistence=kProp["noisePersistence"],\
                  amplitude=kProp["noiseDev"])       
                if kProp["layer"]==None:
                  lVal1D=np.random.normal(kProp["prop"],kProp["interbedPropDev"],(n1Use,))
                else:
                  if not isinstance(kProp["layer"],np.ndArray):
                      raise Exception("layer must be a np.ndarray")
                  loc1D=layer.getNdArray()
                  lVal1D=np.linspace(0,loc1D.shape[0]-1,loc1D.shape[0])
                  nlayerD=loc1D.shape[0]
                  if loc1D.shape[0] >= n1Use:
                    raise Exception("layer is not large enough must be at least %d",n1Use)
                noiseT=createPropArray(nlayer,kProp["depthGrad"]/axes[0].d,\
                  noise3D,layer1D,lVal1D,propV,sendList)
              else:
                raise Exception("Unknown methood to set param ",fld)
          if modify:
            out=outM.getFloatField(fld).getNdArray()
            modifyFloatArray(out,height,noiseT,sinc,expand)
      
      layer=outM.getLayer().getNdArray()
      modifyLayer(layer,height,ievent)
      return outM

          
      




class basicD(deposit):
    def __init__(self,**kw):
        """
        Basic deposit, for now we have not specialized
        
        """
        super().__init__(**kw)




In [ ]:
import numba
import copy 
import skfmm
import datetime
import numpy as np
import random
import multiprocessing
class event:

    def __init__(self,**kw):
        self.kw=kw
    
    def updateArguments(self,**kw):
        """Override arguments used to create class"""

        kuse=copy.deepcopy(self.kw)
        for k,v in kw.items():
            kuse[k]=v
        return  kuse

        
    
    def applyBase(self,**kw):
        """Base class to implement various function, must be overriten"""
        raise Exception("Must override argBase")
    
    def apply(self,modIn:syntheticModel.geoModel,**kw)->syntheticModel.geoModel:
        """
            Basic apply function

            modIn - Input model
        
            You can override in of the arguemnts of the base class with 
            function arguments. Self-doc the base class applyBase function
        
        
        """
        return self.applyBase(modIn,**self.updateArguments(**kw))
@numba.njit(parallel=True)
def buildMap(c2,c3,var,ilayer,dist,sed,sedVal):
    cp=var[c3,c2,0]
    for s in numba.prange(var.shape[0]):
        for m in range(var.shape[1]):
            for f in range(var.shape[2]):
                if abs(var[s,m,f]-cp) < dist and sed[0,f] > sedVal:
                    ilayer[s,m,f]=1
                else:
                    ilayer[s,m,f]=0

@numba.njit(parallel=True)
def fillIndicator(field,thick,ilayer,value):
    for s in numba.prange(field.shape[0]):
        for m in range(field.shape[1]):
            for f in range(thick):
                if ilayer[s,m,f]==1:
                    field[s,m,f]=value
@numba.njit(parallel=True)
def fillFloat(field,thick,ilayer,noise,prop,dev):
    for s in numba.prange(field.shape[0]):
        for m in range(field.shape[1]):
            for f in range(thick):
                if ilayer[s,m,f]==1:
                    field[s,m,f]=prop+dev*noise[s,m,f]    
gl={} 
def initThreads(sh,p,tt,ds):
    gl["shape"]=sh
    gl["phi"]=p
    gl["v_tt"]=tt
    gl["ds"]=ds
def calcFlow(iz):
    sh=gl["shape"]
    p= np.frombuffer(gl["phi"]).reshape((sh[0],sh[1]))
    tt=np.frombuffer(gl["v_tt"]).reshape(sh)
    tt[:,:,iz]=skfmm.travel_time(p,tt[:,:,iz],gl["ds"])
            
class combinedDeposit(event):
    def __init__(self,**kw):
        """Initialize a combined deposit"""
        super().__init__(**kw)
    def applyBase(self,inM:syntheticModel.geoModel,thick:int=30,
      par1:dict={},mixbedFreq=2.,mixbedLacunarity:float=2.,mixbedPersistence=.5,\
      mixbedZStretch:float=1.,mixbedOctaves:int=3,\
      center2:float=.5, center3:float=.5,percFlow:float=20.,\
      seed=None, indicateF:bool=False,\
      indicateI:bool=False,indicateValue=0,\
      prop:dict={},prop_noiseFreq:int=2, prop_noisePersistence:float=.5,\
      prop_noiseOctaves:int=3,prop_noiseZStretch:float=1.,prop_noiseDev:dict={},\
      prop_noiseLacunarity:float=2.,\
      sed_noiseFreq:int=2, sed_noisePersistence:float=.5,\
      sed_noiseOctaves:int=3,sed_noiseZStretch:float=1.,sed_noiseDev:dict={},\
      sed_noiseLacunarity:float=2.,sed_val=-.3,**kw)->syntheticModel.geoModel:
        """Deposit a layer combining two different deposits
        Arguments:
        modIn - Input geomodel
        seed  - [None] - Random seed,set for reproducibility
        thick - [30] - int; Thickness of the layer in number of samples
        par1  - {}    -dict; Parameters for deposit part1
        mixbedFreq     [2.] Higher frequency means faster change
        mixbedLacunarity [2.] How multiple octaves increase in frequency
        mixbedPeristence [.5] How multiple octaves max value changes 
        mixbedZStretch  [1.]  How much to stretch Z axis when calculating layer boundary noise
        mixbedOctaves [3] Number of octaves 
        center2        [.5] Center of deposit2 along axis2
        center3        [.5] Center of deposit3 along axis3 
        percFlow     [20.] Percentile to be covered by flow
        prop    {}     Dictionary containing value for each earth property
        indicateF  [False] Whether or not to mark deposit with float
        indicateI  [False] Whether or not to mark deposit with int
        indicateValue [1]  What to indicate deposit with

        prop_noiseFreq - [2.] Higher frequency means more faster change
        prop_noiseLacunarity [2.] How multiple octaves increase in frequency
        prop_noisePeristence [.5] How multiple octaves max value changes 
        prop_noseOctaves   [3] Number of noise octaves
        prop_noiseZStretch [1.] How much to stretch z axis to create asymetrical noise

        prop_noiseDev: dict or  [0.]  How much deviation to apply to a given property (e.g. vp_noiseDev) 

        sed_noiseFreq - [2.] Higher frequency means more faster change
        sed_noiseLacunarity [2.] How multiple octaves increase in frequency
        sed_noisePeristence [.5] How multiple octaves max value changes 
        sed_noseOctaves   [3] Number of noise octaves
        sed_val            [-.3] Values less than this will be set to sediment

        """


        axes=inM.getPrimaryProperty().getHyper().axes
        exts=[ax.d*ax.n for ax in axes]

        
        maxE=max(max(axes[0].d*thick,exts[1]),exts[2])

        y=np.linspace(0,3.*exts[1]/maxE,axes[1].n,dtype=np.float32)
        x=np.linspace(0.,3.*exts[2]/maxE,axes[2].n,dtype=np.float32)
        zL=np.linspace(0,3.*thick*axes[0].d/maxE*mixbedZStretch,thick,dtype=np.float32)

        if seed is None:
            seed =int(datetime.datetime.utcnow().timestamp())
        np.random.seed(seed)
        random.seed(seed)

        noiseG=simplexNoise.simplexNoise(random.randint(0,1000*1000*1000)) 
        noiseV=noiseG.noise3D(zL,y,x,frequency=mixbedFreq,lucanarity=mixbedLacunarity,\
        persistence=mixbedPersistence,n_octaves=mixbedOctaves,amplitude=1)
        
        sedG=simplexNoise.simplexNoise(random.randint(0,1000*1000*1000)) 
        sedV=sedG.noise2D(zL,x,frequency=sed_noiseFreq,lucanarity=sed_noiseLacunarity,\
        persistence=sed_noisePersistence,n_octaves=sed_noiseOctaves,amplitude=1)
        
        
        phi=np.ones((noiseV.shape[0],noiseV.shape[1]))
        noiseV=2+3.985*((noiseV-noiseV.min())/(noiseV.max()-noiseV.min())-.5)
        c2,c3=int(center2*axes[1].n+.5),int(center3*axes[1].n+.5)
        phi[c3:c3+1,c2:c2+1]=-1
        
        sh=noiseV.shape
        p=multiprocessing.RawArray("d",sh[0]*sh[1])
        pnp=np.frombuffer(p).reshape((sh[0],sh[1]))
        np.copyto(pnp,phi)
        vt=multiprocessing.RawArray("d",sh[0]*sh[1]*sh[2])
        vtnp=np.frombuffer(vt).reshape(sh)
        np.copyto(vtnp,noiseV)
        print("before multiprocessing")
        with multiprocessing.Pool(processes=max(1,min(int(noiseV.shape[2]/2),multiprocessing.cpu_count())),\
        initializer=initThreads(noiseV.shape,p,vt,.005)) as pool:
            pool.map(calcFlow, range(noiseV.shape[2]))
            pool.close()
            pool.join()

        print("after multi")    
        par1["thick"]=thick

        outM=basicD().apply(inM,**par1)

        ilayer=np.zeros((axes[2].n,axes[1].n,thick),dtype=np.int32)

        dist=np.percentile(vtnp,percFlow)
        buildMap(c2,c3,vtnp,ilayer,dist,sedV,sed_val)
        zL=np.linspace(0,3.*thick*axes[0].d/maxE*prop_noiseZStretch,thick,dtype=np.float32)
        noiseG=simplexNoise.simplexNoise(random.randint(0,1000*1000*1000)) 
        noiseV=noiseG.noise3D(zL,y,x,frequency=prop_noiseFreq,lucanarity=prop_noiseLacunarity,\
        persistence=prop_noisePersistence,n_octaves=prop_noiseOctaves,amplitude=1)
        for fld in outM.getFloatFieldList():
            if fld != "indicate":
                if not fld in prop:
                    raise Exception(f"No property specified for fld={fld}")
                dev=0
                if fld in prop_noiseDev:
                    dev=prop_noiseDev[fld]
                    dev=0
                f=outM.getFloatField(fld).getNdArray()
                fillFloat(f,thick,ilayer,noiseV,prop[fld],dev)
        if indicateF:
            ind=outM.getCreateFloatField("indicateor",0.,0.).getNdArray()
            fillIndictor(ind,thick,ilayer,indicateValue)
        if indicateI:
            ind=outM.getCreateIntField("indicator",0,0).getNdArray()
            fillIndicator(ind,thick,ilayer,indicateValue)
        layer=outM.getIntField("layer").getNdArray()
        fillIndicator(layer,thick,ilayer,outM.getNewEvent())
        return outM,vtnp




In [ ]:
pars=dict(noiseDev=120,interbedThick=15., interbedThickDev=13.,noiseFrequency=3,\
          interbedPropDev=100.,prop=3000,interbedDev=4,seed=3)
update=combinedDeposit()
nt=3
thicks=np.random.uniform(low=10,high=60,size=(nt))

thicks=[3,20,3]
center2=np.random.uniform(low=0.3,high=.7,size=(nt))
center3=np.random.uniform(low=0.3,high=.7,size=(nt))
freqs=np.random.uniform(low=0.3,high=.7,size=(nt))
zst=np.random.uniform(low=10,high=50,size=(nt))
mixF=np.random.uniform(low=0.1,high=.3,size=(nt))
zst2=np.random.uniform(low=8,high=13,size=(nt))
flows=np.random.uniform(low=23.,high=40,size=(nt))
flows=[60,30,70]
newS=syntheticModel.geoModel(hyper,{"vp":3000.},"vp")
newS=depositI.apply(newS,thick=90,prop=3340)

for i in range(nt):
    print(i)
    newS,vv=update.applyBase(newS,thick=int(thicks[i]),mixbedZStretch=zst[i],\
    mixbedFreq=freqs[i],par1=pars,prop=dict(vp=6000),\
    prop_noiseZStretch=zst2[i],center2=center2[i],center3=center3[i],\
     sedVal=-.45, percFlow=flows[i],prop_noiseDev={"vp":100.})
newS=syntheticModel.squish(maxShift=120,begFreq=8).apply(newS)
newS=depositI.apply(newS,prop=2600,thick=20)
newS=depositC.apply(newS,prop=1450,thick=40)
plt=createPlot(newS.get(),40,400,400)
plt

In [ ]:
plt=createPlot(newS.get(),96,400,400)
plt

#### plt=createPlot(update.getFloatField("rho"),40,400,400)
plt

In [ ]:
import syntheticModel.event as event
import syntheticModel.sincTable as sincTable
import syntheticModel.syntheticModel as syntheticModel
from syntheticModel.model_noise import fractal
from numba import jit, njit, prange, float64
import numpy as np
import datetime
import math

RAND_INT_LIMIT = 100000
FREQUENCY_LEVELS = [3, 6, 12, 24]
AMPLITUDE_LEVELS = [1, 0.5, 0.125, 0.0625]
#@njit(float64[:,:](float,float64[:,:],int,int),parallel=True)
@njit(parallel=True)
def rotateField(azimuth:float,field:np.ndarray,n1:int,n2:int):
    cIn=int(field.shape[0]/2.)
    cOut1=int(n1/2)
    cOut2=int(n2/2)
    rot=azimuth/180*math.pi
    out=np.zeros((n2,n1))
    for i2 in prange(n2):
        f2=(i2-cOut2)
        for i1 in range(n1):
            f1=(i1-cOut1)
            b1=math.cos(rot)*f1-f2*math.sin(rot)+cIn
            b2=math.cos(rot)*f2+f1*math.sin(rot)+cIn
            il1=int(b1)
            il2=int(b2)
            b1-=il1
            b2-=il2

            out[i2,i1]=(1.-b1)*(1.-b2)*field[il2,il1]+\
              (   b1)*(1.-b2)*field[il2,il1+1]+\
              (1.-b1)*(   b2)*field[il2+1,il1]+\
              (   b1)*(   b2)*field[il2+1,il1+1]

            #out[i2,i1]=field[il2,il1]
    return out

            
    


def getNoiseField(n1,n2,maxS,widthInLine,widthCrossLine,azimuth,noise_levels,begFreq):

    nmax=int(max(n1,n2)*1.8)
    # randomly shift origin
    o1 = np.random.rand() * nmax
    o2 = np.random.rand() * nmax
    p1 = np.linspace(o1, o1 + nmax*widthInLine, nmax, endpoint=False)
    p2 = np.linspace(o2, o2 + nmax*widthCrossLine , nmax, endpoint=False)
    p = np.stack(np.meshgrid(p1, p2, indexing='ij'), -1)
    p = np.reshape(p, (-1, 2)).T

    # get freq and amplitude values for perlin octaves
    freq=[begFreq]
    for i in range (3):
        freq.append(freq[-1]*2)

    freq_levels = list(np.array(freq[:noise_levels]) / (n1 ))
    amp_levels = AMPLITUDE_LEVELS[:noise_levels]
    field = fractal(p,
                    frequency=freq_levels,
                    n_octaves=noise_levels,
                    amplitude=amp_levels)
    field = np.reshape(field, (nmax, nmax))

    # normalize between -1 and 1
    mx=np.max(field)
    mn=np.min(field)
    field = (field+mn)/(mx-mn) *maxS
    m=rotateField(azimuth,field,n1,n2)
    return m


@njit(parallel=True)
def shiftFieldF(maxS:int,sincT:np.ndarray,outF:np.ndarray,inF:np.ndarray,shiftF:np.ndarray,\
               fill:float,basement:float):
    for i3 in prange(outF.shape[0]):
        for i2 in prange(outF.shape[1]):
            useF=maxS-shiftF[i3,i2]
            imin=math.ceil(useF)
            itab=min(sincT.shape[0]-1,int((imin-useF)*sincT.shape[0]+.5))
            
            for i1 in range(outF.shape[2]):
                #outF[i3,i2,i1]=inF[i3,i2,max]
                outF[i3,i2,i1]=0
                for isinc in range(sincT.shape[1]):
                    il=min(inF.shape[2]-1,max(0,i1-3+isinc-imin))
                    outF[i3,i2,i1]+=\
                    inF[i3,i2,il]*sincT[itab,isinc]
                if i3==490 and i2==709 and i1==200:
                    print(i1,imin,itab,outF[i3,i2,i1])


@njit(parallel=True)
def shiftFieldI(maxS:int,outF:np.ndarray,inF:np.ndarray,shiftF:np.ndarray,\
               fill:float,basement:float):
    for i3 in prange(outF.shape[0]):
        for i2 in prange(outF.shape[1]):
            imin=int(maxS-shiftF[i3,i2]+.5)
            for i1 in range(imin):
                outF[i3,i2,i1]=fill
            for i1 in range(inF.shape[2]):
                outF[i3,i2,i1+imin]=inF[i3,i2,i1]
            for i1 in range(inF.shape[2]+imin,outF.shape[2]):
                outF[i3,i2,i1]=basement            
class squish(event.event):
    """Default class for squishing"""
    def __init__(self,**kw):
        super().__init__(**kw)
    def applyBase(self,inM:syntheticModel.geoModel,azimuth:float=0.,\
        widthInLine:float=1000., widthCrossLine:float=100.,\
        maxShift:float=50, seed=None,begFreq:float=6.,nfreq=3)->syntheticModel.geoModel:
        """

        Squish a model

        Arguements:

        inM - Input model
        azimuth - Azimuth for squishing
        widthInLine - Approximate width for random patterns, axis 1
        widthCrosssLine - Approximate width for random patterns, axis 2
        maxShift -  Maximum shift in z
        seed - [null] int; random seed to initialize noise functions
        nfreq -3 Number of frequencies for noise (more means higher frequency bumpiness)
        begFreq - 3. Basic frequency level for noise (lower means lowe spatial frequency)

        Returns

            outModel - Returns updated model  
        """
        if seed is None:
            seed =int(datetime.datetime.utcnow().timestamp())
        np.random.seed(seed)

        axes=inM.getPrimaryProperty().getHyper().axes

        #calculate the maximum shift and then add cells to the top
        maxS=math.ceil(maxShift/axes[0].d)
        outM=inM.expand(maxS,0)

        sincT=sincTable.table

        shifts=getNoiseField(axes[1].n,axes[2].n,maxS,widthInLine/axes[1].n/axes[1].d,\
            widthCrossLine/axes[2].d/axes[2].n,azimuth, nfreq,begFreq)
        shifts-=shifts.min()
        self.shifts=shifts


        for fld in outM.getFloatFieldList():
            inp=inM.getFloatField(fld).getNdArray()
            base=inM.getBasementFloat(fld)
            outp=outM.getFloatField(fld).getNdArray()

            shiftFieldF(maxS,sincT,outp,\
                        inp,shifts,inM.getFillFloat(fld),base)

        for fld in outM.getIntFieldList():
            inp=inM.getIntField(fld).getNdArray()
            base=inM.getBasementInt(fld)
            
            shiftFieldI(maxS,outM.getIntField(fld).getNdArray(),\
                        inp,shifts,base,inM.getFillInt(fld))
        return outM




class basic(squish):
    def __init__(self,**kw):
        """
        Basic squish, for now we have not specialized
        
        """
        super().__init__(**kw)		

In [ ]:
squished2=basic().apply(update,begFreq=24,maxShift=470,seed=55)
#squished2=depositIC.apply(squished2,thick=70, prop=1450,seed=33)

plt=createPlot(squished2.get(),120,340,490)
plt

Lets deposit another layer

Now lets create a squish event

Lets create a bunch of faults.

In [ ]:
dep3=depositC.apply(squished2,thick=1, prop=1450,seed=58)
np.random.seed(99)
nb=1
ns=1
xs=np.random.uniform(low=.1,high=.9,size=(nb,ns))
ys=np.random.uniform(low=.1,high=.9,size=(nb,ns))
zs=np.random.uniform(low=.1,high=.9,size=(nb,ns))
angles=np.random.uniform(low=32,high=44,size=(nb,ns))
rads=np.random.uniform(low=1850,high=2150,size=(nb,ns))
shifts=np.random.uniform(low=130,high=160,size=(nb,ns))
lengths=np.random.uniform(low=300,high=440,size=(nb,ns))
azimuth=np.random.uniform(low=12,high=23,size=(nb,ns))
seeds=np.random.randint(99,size=(nb,ns))



In [ ]:
import syntheticModel.fault as fault
dep3=depositC.apply(squished2,thick=1, prop=1850,seed=86)

for i in range(xs.shape[0]):
    for j in range(xs.shape[1]):
        print("working on",j,i)
        dep3=syntheticModel.fault().apply(dep3,angle=angles[i,j],radius=rads[i,j],begx=xs[i,j],begy=ys[i,j],\
                            azimuth=azimuth[i,j],seed=seeds[i,j],\
        begz=zs[i,j],indicateI=True,shift=shifts[i,j],ruptureLength=lengths[i,j])
        print(f"beg={zs[i,j]},{ys[i,j]},{xs[i,j]} angle={angles[i,j]}, {lengths[i,j]}")
    #dep3=depositC.apply(dep3,thick=1,prop=1850)

    
    
plt=createPlot(dep3.get(),120,340,490)
plt

In [ ]:

s=squished2.get().getNdArray()
print(s.min(),s.max())
plt=createPlot(squished2.get(),200,340,490)
plt

n=s[490,709,200]
print(n)
